In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

BATCHES_COUNT = 12091

In [2]:
%%time
inc_store = pd.HDFStore('../input/se_inc_test.h5')
inc_keys = inc_store.keys()
print('{} tables in storage'.format(len(inc_keys)))

12091 tables in storage
CPU times: user 44.1 s, sys: 1.13 s, total: 45.2 s
Wall time: 54.3 s


In [3]:
TEST_SIZE = 1768182

ave_preds = []
cur_id = None
cur_sum = None

for key in tqdm(inc_keys): 
    inc_table = inc_store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    for i in range(inc_table.shape[0]):
        product_id = inc_table.pr_id.iloc[i]
        
        #res_sum1 = res_table.iloc[pd.np.r_[st : st + 4, st + 5:st + 9]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        #res_sum2 = res_table.iloc[pd.np.r_[st + 4: st + 5, st + 9:st + 10]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        #dens_sum1 = dens_table.iloc[pd.np.r_[st : st + 4, st + 5:st + 9]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        #dens_sum2 =dens_table.iloc[pd.np.r_[st + 4: st + 5, st + 9:st + 10]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        new_sum = inc_table[i : i + 1].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        #new_sum = res_sum + dens_sum + inc_sum
        #new_sum = res_sum1 + 3*res_sum2 + dens_sum1 + 3*dens_sum2
        if cur_id == product_id:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            ave_preds.append((int(cur_id), cat))
            
        cur_id = product_id
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    ave_preds.append((int(cur_id), cat))
            
ave_preds_df = pd.DataFrame(data=ave_preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(ave_preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(ave_preds_df.shape[0] - TEST_SIZE,
                                                float(ave_preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
ave_no_dupls = ave_preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape:', ave_no_dupls.shape)

ave_no_dupls.to_csv('../submit/se_inc.csv', index=False)

100%|██████████| 12091/12091 [1:32:33<00:00,  2.18it/s]


Dataframe shape (1769197, 2); products in test 1768182
Duplicates 1015; affect accuracy 0.0005740359306903928
No duplicated shape: (1768182, 2)
